# 텍스트 로드하기

이번 튜토리얼에서는 텍스트를 로드하고 전처리하는 두 가지 방법을 소개합니다.
+ keras 유틸리티와 레이어 사용하기
+ 텍스트 파일을 로드하기 위해 `tf.data.TextLineDataset`를 사용하고, 데이터를 전처리하기 위해 `tf.text`를 사용하기

In [1]:
!pip uninstall -y tensorflow tf-nightly keras

!pip install -q -U tf-nightly
!pip install -q -U tensorflow-text-nightly

Uninstalling tf-nightly-2.7.0.dev20210713:
  Successfully uninstalled tf-nightly-2.7.0.dev20210713


In [2]:
import collections, pathlib, re, string

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

## 1. 스택 오버플로우를 위한 태그 예측하기

스택 오버플로우로부터 프로그래밍 질문 데이터세트를 다운로드합니다. 각 질문(어떻게 딕셔너리를 값에 따라 정렬하는가?)은 하나의 태그(`Python`, `CSharp`, `JavaScript`, `Java` 중 하나)에 레이블되어 있습니다.

이번 과제는 질문에 대한 태그를 예측하는 모델을 개발하는 멀티 클래스 분류 작업입니다.

### 데이터세트 다운로드 및 살펴보기

이제 데이터세트를 다운로드하고 딕셔너리 구조를 살펴봅시다.

In [3]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
dataset_dir = tf.keras.utils.get_file(
    origin=data_url,
    untar=True,
    cache_dir='stack_overflow',
    cache_subdir=''
)

# 경로를 PosixPath로 나타내도록 한다. .parent는 가장 마지막 파일 바로 위까지의 경로를 나타낸다.
dataset_dir = pathlib.Path(dataset_dir).parent

In [4]:
list(dataset_dir.iterdir())

[PosixPath('/tmp/.keras/test'),
 PosixPath('/tmp/.keras/stack_overflow_16k.tar.gz'),
 PosixPath('/tmp/.keras/train'),
 PosixPath('/tmp/.keras/README.md')]

In [5]:
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

[PosixPath('/tmp/.keras/train/python'),
 PosixPath('/tmp/.keras/train/javascript'),
 PosixPath('/tmp/.keras/train/csharp'),
 PosixPath('/tmp/.keras/train/java')]

`train/csharp`, `train/java`, `train/python`, `train/javascript` 디렉토리에는 많은 텍스트 파일들이 저장되어 있습니다. 각 파일들은 스택 오버플로우 질문들입니다. 파일 하나의 데이터를 살펴봅시다.

In [6]:
sample_file = train_dir/'python/1755.txt'
print(open(sample_file).read())

why does this blank program print true x=true.def stupid():.    x=false.stupid().print x



### 데이터세트 로드 및 구성하기

이제 디스크에서 데이터를 로드하고 훈련에 적합한 포맷으로 만들 것입니다. `tf.data.Dataset`을 생성하기 위해 `text_dataset_from_directory`를 사용하겠습니다. `tf.data`는 입력 파이프라인을 빌드하는데 아주 강력한 툴입니다.

`preprocessing.text_dataset_from_directory`는 아래와 같은 디렉토리 구조를 가집니다.

```
train/
...csharp/
......1.txt
......2.txt
...java/
......1.txt
......2.txt
...javascript/
......1.txt
......2.txt
...python/
......1.txt
......2.txt
```

머신 러닝 실험을 진행할 때, 데이터세트를 훈련, 검증, 테스트의 3가지로 나누는 것이 가장 좋습니다. 스택 오버플로우 데이터세트는 이미 훈련과 테스트가 나뉘어져 있으므로, 훈련 데이터세트에서 8:2 비율로 검증 세트를 하나 만들겠습니다. `validation_split`을 이용합니다.

In [7]:
batch_size = 32
seed = 42

raw_train_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size = batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


위에서 보았듯, 8,000개의 샘플들이 훈련 폴더 내에 있고 그 중 80%인 6,400개가 훈련 데이터세트가 된 것을 확인할 수 있습니다. 6,400개의 데이터는 각각 32개씩 배치(데이터 묶음)을 이루고 있습니다.

훈련 데이터세트는 이후에 `tf.data.Dataset`을 거쳐 `model.fit`으로 들어가 학습하는 용도로 쓰이게 됩니다.

훈련 데이터세트의 example과 label을 살펴봅시다.

In [8]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(10):
        print("Question: ", text_batch.numpy()[i])
        print('Label: ', label_batch.numpy()[i])

Question:  b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default con

첫 번째 배치의 32개의 데이터 중 10개의 데이터를 출력해보았습니다. 

레이블이 0, 1, 2, 3으로 나타나는데 각 레이블이 의미하는 언어를 알아보겠습니다.

In [9]:
for i, label in enumerate(raw_train_ds.class_names):
    print("Label", i, "corresponds to", label)

Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


이제 검증 데이터세트를 만들어보겠습니다.

주의: `validation_split`과 `subset`을 사용할 때 반드시 랜덤 seed를 명시하거나 `shuffle=False`을 적어주어, 훈련 데이터세트와 검증 데이터세트가 서로 겹치지 않도록 해야합니다.

In [10]:
raw_val_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size = batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


마지막으로 훈련 데이터세트를 만들어 보겠습니다.

In [11]:
test_dir = dataset_dir/'test'
raw_test_ds = preprocessing.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size
)

Found 8000 files belonging to 4 classes.


### 훈련을 위한 데이터세트 준비하기(전처리 작업)

이제 `preprocessing.TextVectorization` 레이어에 사용할 데이터를 표준화, 토큰화, 벡터화하는 전처리 작업을 거쳐야합니다.

+ 표준화(Standardization) : 구두점 또는 HTML의 요소들을 제거해서 데이터세트를 단순화하는 텍스트 전처리 과정.
+ 토큰화(Tokenization) : 문자열을 토큰으로 나누는 과정(예를 들어, 문장을 띄어쓰기에 따라 나눠서 단어로 만드는 과정이 이에 속합니다)
+ 벡터화(Vectorization) : 나누어진 토큰들을 숫자로 변환하는 과정입니다. 벡터화된 숫자들이 신경망에 input으로 입력됩니다.

각 과정들은 아래와 같이 레이어됩니다.

+ 디폴트 표준화 작업은 텍스트를 소문자로 변환하고 구두점을 제거합니다.
+ 디폴트 토큰화 작업은 공백에 따라 문장을 분리합니다.
+ 디폴트 벡터화 모드는 `int`입니다. 각 정수는 하나의 단어를 가리킵니다. `binary`라는 다른 모드를 사용할 수도 있습니다. 이 모드는 bag-of-word 모델을 빌드하기 위함입니다.   

벡터화 모드의 `binary`와 `int` 두 가지 모드를 모두 배워보겠습니다. 먼저 `binary` 모드를 이용해 bag-of-words 모델을 빌드해본 뒤에 `int` 모드를 `1D ConvNet`과 함께 사용해보겠습니다.

먼저 각 `binary`와 `int` 벡터화 레이어를 만든 뒤에 `adapt`를 이용해 훈련 될 수 있도록 합니다.

In [12]:
VOCAB_SIZE = 10000
# binary layer
binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary'
)

`int` 모드에서는 maximum vocabulary size와 더불어 maximum sequence length도 지정해주어야 합니다. maximum length를 벗어나는 문장의 경우 앞 또는 뒤에서부터 문장을 자릅니다. maximum length보다 부족한 길이일 경우 부족한만큼 0으로 채웁니다.

In [13]:
MAX_SEQUENCE_LENGTH = 250
# int layer
int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH
)

다음으로 `adapt`를 콜해서 전처리 레이어의 상태를 데이터세트를 이용해 훈련시켜야합니다. 이 과정은 모델이 문자에 해당하는 정수를 빌드하도록 즉, 벡터화 해줍니다.

주의: `adapt`를 콜할 때 반드시 훈련 데이터만을 사용해야합니다.

In [14]:
train_text = raw_train_ds.map(lambda text, label: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

데이터를 전처리하기 위해 이 레이어들을 이용한 결과를 살펴봅시다.

[tf.expand_dims](https://www.tensorflow.org/api_docs/python/tf/expand_dims)

In [15]:
def binary_vectorize_text(text, label):
    text = tf.expand_dims(text, -1)  # text의 마지막에 1을 추가한다.
    return binary_vectorize_layer(text), label

In [16]:
def int_vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return int_vectorize_layer(text), label

In [17]:
# 하나의 배치(32개의 데이터)를 데이터세트로부터 뽑아냅니다.
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question: ", first_question)
print('Label: ', first_label)

Question:  tf.Tensor(b'"what is the difference between these two ways to create an element? var a = document.createelement(\'div\');..a.id = ""mydiv"";...and..var a = document.createelement(\'div\').id = ""mydiv"";...what is the difference between them such that the first one works and the second one doesn\'t?"\n', shape=(), dtype=string)
Label:  tf.Tensor(2, shape=(), dtype=int32)


In [18]:
print("'binary' vectorized question:", binary_vectorize_text(first_question, first_label)[0])

'binary' vectorized question: tf.Tensor([[1. 1. 0. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [19]:
print("'int' vectorized question:", int_vectorize_text(first_question, first_label)[0])

'int' vectorized question: tf.Tensor(
[[ 55   6   2 410 211 229 121 895   4 124  32 245  43   5   1   1   5   1
    1   6   2 410 211 191 318  14   2  98  71 188   8   2 199  71 178   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0


위에서 보았듯이 `binary` 모드는 maximum vocabulary size내에 속해 있는지 여부를 0과 1로 나타내는 반면, `int` 모드는 각 토큰을 해당하는 정수로 변환해서 질서를 보존합니다.

`.get_vocabulary()` 메서드를 통해 정수에 대응되는 문자가 무엇인지 알 수 있습니다.

In [20]:
print('1289 ---> ', int_vectorize_layer.get_vocabulary()[1289])
print('313 ---> ', int_vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(int_vectorize_layer.get_vocabulary())))

1289 --->  roman
313 --->  source
Vocabulary size: 10000


이제 모델을 훈련시킬 준비가 거의 끝났습니다. 마지막 전처리 과정으로 `TextVectorization` 레이어를 훈련, 검증, 테스트 데이터세트에 적용합니다.

In [21]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

### 훈련 성능 향상시키기

I/O가 블로킹되지 않도록 데이터를 로딩할 때 필수적으로 해야할 2가지가 있습니다.

+ `.cache()` : 디스크로부터 데이터를 로드한 후에 메모리에 남겨놓도록 합니다. 모델을 훈련할 때 데이터세트가 소위 병목(병의 목 부분처럼 좁아서 흐름이 지연되는 현상) 현상이 되는 것을 방지할 수 있습니다. 데이터세트의 크기가 너무 커서 메모리 안에 훈련될 수 없다면, 이 방법이 작은 파일들을 읽는 것보다 훨씬 효율적으로 데이터를 읽어들일 수 있는 캐시를 생성해서 해결할 수 있습니다.

+ `.prefetch()` : 훈련하는 동안 데이터의 전처리 과정과 모델 학습 작용이 동시에 이루어지도록 합니다.

In [22]:
AUTOTUNE = tf.data.AUTOTUNE  # 컴퓨터의 CPU에 맞는 적절한 구동 코어수 결정
# 데이터 세트를 캐시 및 프리패치하는 함수 생성
def configure_dataset(dataset):
    return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [23]:
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

### 모델 훈련하기

이제 신경망을 구성할 차례입니다.

`binary` 벡터화 데이터를 이용해 간단한 bag-of-words 선형 모델을 학습해봅시다.

In [24]:
binary_model = tf.keras.Sequential([layers.Dense(4)])
binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)
history = binary_model.fit(
    binary_train_ds, 
    validation_data=binary_val_ds, 
    epochs=10
)

Epoch 1/10
200/200 [==============================] - 3s 15ms/step - loss: 1.1237 - accuracy: 0.6433 - val_loss: 0.9194 - val_accuracy: 0.7788
Epoch 2/10
200/200 [==============================] - 1s 4ms/step - loss: 0.7807 - accuracy: 0.8206 - val_loss: 0.7538 - val_accuracy: 0.8006
Epoch 3/10
200/200 [==============================] - 1s 4ms/step - loss: 0.6288 - accuracy: 0.8592 - val_loss: 0.6675 - val_accuracy: 0.8150
Epoch 4/10
200/200 [==============================] - 1s 4ms/step - loss: 0.5351 - accuracy: 0.8845 - val_loss: 0.6137 - val_accuracy: 0.8219
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4690 - accuracy: 0.9022 - val_loss: 0.5767 - val_accuracy: 0.8313
Epoch 6/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4186 - accuracy: 0.9175 - val_loss: 0.5499 - val_accuracy: 0.8319
Epoch 7/10
200/200 [==============================] - 1s 4ms/step - loss: 0.3783 - accuracy: 0.9270 - val_loss: 0.5297 - val_accuracy: 0.8350
Epoch

이제 `int` 벡터화 레이어를 사용해 `1D ConvNet`을 빌드해봅시다.

In [25]:
def create_model(vocab_size, num_labels):
    model = tf.keras.Sequential([
                                 layers.Embedding(vocab_size, 64, mask_zero=True),
                                 layers.Conv1D(64, 5, padding='valid', activation='relu', strides=2),
                                 layers.GlobalMaxPooling1D(),
                                 layers.Dense(num_labels)
    ])
    return model

In [26]:
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)

Epoch 1/5
200/200 [==============================] - 9s 41ms/step - loss: 1.1398 - accuracy: 0.5200 - val_loss: 0.7533 - val_accuracy: 0.6862
Epoch 2/5
200/200 [==============================] - 6s 31ms/step - loss: 0.6188 - accuracy: 0.7597 - val_loss: 0.5419 - val_accuracy: 0.7987
Epoch 3/5
200/200 [==============================] - 6s 31ms/step - loss: 0.3758 - accuracy: 0.8806 - val_loss: 0.4778 - val_accuracy: 0.8219
Epoch 4/5
200/200 [==============================] - 6s 30ms/step - loss: 0.2109 - accuracy: 0.9511 - val_loss: 0.4729 - val_accuracy: 0.8213
Epoch 5/5
200/200 [==============================] - 6s 30ms/step - loss: 0.1067 - accuracy: 0.9831 - val_loss: 0.4939 - val_accuracy: 0.8194


두 모델을 비교해봅시다.

In [27]:
print('Linear model on binary vectorized data:')
print(binary_model.summary())

Linear model on binary vectorized data:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())

ConvNet model on int vectorized data:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640064    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d (Globa  (None, 64)                0         
lMaxPooling1D)                                                   
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 660,868
Trainable params: 660,868
Non-trainable params: 0
_________________________________________________________________
None


두 모델을 테스트 데이터를 이용해 평가해봅시다.

In [29]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))

250/250 [==============================] - 4s 14ms/step - loss: 0.5196 - accuracy: 0.8075
Binary model accuracy: 81.42%
Int model accuracy: 80.75%


주의: 이 데이터세트는 다소 간단한 분류 문제를 보여줍니다. 더 복잡한 데이터세트와 문제들은 전처리 전략과 모델 아키텍처에 따라서 사소하지만 중요한 변화를 가져옵니다. 여러 다른 하이퍼파라미터와 에포크를 시도해보면서 다양한 접근을 해보는 것이 중요합니다.

### 모델 익스포트하기

앞서 텍스트를 모델에 입력하기 전에 `TextVectorization` 레이어를 적용했습니다. 만약 전처리 되지 않은 문자열을 바로 모델에 입력하고 싶다면, `TextVectorization` 레이어를 모델 안에 넣으면 됩니다.

In [30]:
export_model = tf.keras.Sequential([
                                    binary_vectorize_layer,
                                    binary_model,
                                    layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {:2.2%}".format(accuracy))

/usr/local/lib/python3.7/dist-packages/keras/backend.py:4908: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


250/250 [==============================] - 3s 10ms/step - loss: 0.5182 - accuracy: 0.8142
Accuracy: 81.42%


이미 `binary_vectorize_layer`와 `binary_model`은 앞서 훈련이 되었으므로 새로 만들어진 `export_model`을 따로 훈련하진 않았습니다.

이렇게 함으로써 모델에 전처리 되지 않은 문자열을 입력으로 넣고 `model.predict`를 사용해 각 레이블에 대한 확률을 예측할 수 있습니다.

최댓값을 가지는 레이블을 찾는 함수를 정의해봅시다.

In [31]:
def get_string_labels(predicted_scores_batch):
    predicted_int_labels = tf.argmax(predicted_scores_batch, axis=1)
    predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
    return predicted_labels

### 새로운 데이터를 사용해 결과 예측하기

In [32]:
inputs = [
          "how do I extract keys from a dict into a list?",  # python
          "debug public static void main(string[] args) {...}",  # java
]
predicted_scores = export_model.predict(inputs)
predicted_labels = get_string_labels(predicted_scores)
for input, label in zip(inputs, predicted_labels):
    print("Question: ", input)
    print("Predicted label: ", label.numpy())

Question:  how do I extract keys from a dict into a list?
Predicted label:  b'python'
Question:  debug public static void main(string[] args) {...}
Predicted label:  b'java'


텍스트 전처리 로직을 모델 안에 포함하는 것은 새로운 데이터를 예측할 때 훨씬 단순화하고 잠재적인 [train/test skew](https://developers.google.com/machine-learning/guides/rules-of-ml#training-serving_skew)를 감소시켜줍니다.

`TextVectorization` 레이어를 어디에 적용시키느냐에 따라 성능에 차이가 생긴다는 점을 명심해야합니다. 이 레이어를 모델 밖에서 사용하면 CPU 처리와 GPU에서의 훈련 데이터 버퍼링이 동시에 일어나지 않습니다. 그래서 GPU에서 모델을 훈련한다면, 모델의 좋은 훈련 성능을 위한 선택이 될 수 있습니다. 그리고 나서 `TextVectorization` 레이어를 모델 안으로 위치를 바꿔주면 앞으로 새로운 데이터를 예측할 때 좋은 성능을 낼 수 있을 것입니다.

## 2. 일리아드 번역 저자 예측하기

이번에는 `tf.data.TextLineDataset`을 이용해 텍스트 파일로부터 example을 로드하고 `tf.text`를 이용해 데이터를 전처리 할 것입니다. example은 Homer의 일리아드라는 작품을 영문으로 번역한 3개의 번역본입니다. 텍스트 한 줄이 주어졌을 때, 번역가가 누구인지를 판별하는 모델을 훈련시킬 것입니다.

### 데이터세트 다운로드 및 살펴보기

3명의 번역가는 아래와 같습니다.
- [William Cowper](https://en.wikipedia.org/wiki/William_Cowper) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)

- [Edward, Earl of Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)

- [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

텍스트 파일은 이미 헤더, 풋터, 라인 넘버, 챕터 타이틀을 제거하는 전처리 작업을 거쳤습니다. 변환된 파일들을 다운로드합니다.

In [33]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
    text_dir = utils.get_file(name, origin=DIRECTORY_URL + name)

parent_dir = pathlib.Path(text_dir).parent
list(parent_dir.iterdir())

[PosixPath('/root/.keras/datasets/butler.txt'),
 PosixPath('/root/.keras/datasets/derby.txt'),
 PosixPath('/root/.keras/datasets/cowper.txt')]

### 데이터세트 로드하기

`TextLineDataset`는 오리지널 파일에서 텍스트를 한 줄 씩 example로 만들어서 `tf.data.Dataset`을 생성하도록 고안되었습니다. 반면에 `text_dataset_from_directory`는 파일 전체의 내용을 하나의 example로 만듭니다. `TextLineDataset`은 줄 단위로 되어있는 텍스트 데이터를 다루는데 유용합니다.

for문으로 반복처리하면서 3개의 텍스트 파일에서 데이터세트를 로드합니다. 각 example은 각각의 레이블에 대응되어야하므로 `tf.data.Dataset.map`을 이용해서 (example, label) 쌍을 줄 것입니다.

In [34]:
def labeler(example, index):
    return example, tf.cast(index, tf.int64)

In [35]:
labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
    # 3개의 파일을 TextLineDataset 처리해줍니다.
    lines_dataset = tf.data.TextLineDataset(str(parent_dir/file_name))
    # (example, label) 쌍을 만듭니다.
    labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
    # labeled_data_sets에 추가합니다.
    labeled_data_sets.append(labeled_dataset)

이제 레이블된 데이터세트들을 결합해서 하나의 데이터세트로 만들고 섞어줄 것입니다.

In [36]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
VALIDATION_SIZE = 5000

In [37]:
all_labeled_data = labeled_data_sets[0]  # 첫 번째 파일의 데이터세트를 먼저 넣어줍니다.
for labeled_dataset in labeled_data_sets[1:]:  # 두 번째 파일부터 반복하며 concatenate를 이용해 데이터세트를 추가합니다.
    all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
# 결합된 데이터세트 내의 데이터들을 한 번 섞어줍니다.
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

하나가 된 데이터세트 내의 10개의 데이터를 살펴보겠습니다.

In [38]:
for text, label in all_labeled_data.take(10):
    print("Sentence: ", text.numpy())
    print("Label: ", label.numpy())

Sentence:  b'The dust, put on their tunics. Then again'
Label:  0
Sentence:  b"But miss'd Patroclus; the innocuous point,"
Label:  0
Sentence:  b"He said; and from th' applauding ranks of Greece"
Label:  1
Sentence:  b'The Gods, or our supineness, succor Troy.'
Label:  0
Sentence:  b"And struck the circle of AEneas' shield"
Label:  1
Sentence:  b'Of Agamemnon, where the Chiefs, perchance,'
Label:  0
Sentence:  b'For him, with confident persuasion all'
Label:  0
Sentence:  b"His mother, Juno, white-arm'd Queen of Heav'n:"
Label:  1
Sentence:  b'the Trojans. Therefore he drew back, and the Trojans flung fire upon'
Label:  2
Sentence:  b'Noblest of all our host! bear with my grief,'
Label:  0


### 훈련을 위한 데이터세트 준비

Keras의 `TextVectorization` 레이어를 이용해 텍스트 데이터세트를 전처리하는 대신, 이번에는 `tf.text` API를 이용해 텍스트 데이터를 표준화 및 토큰화하고, 단어 사전을 빌드한 후 `StaticVocabularyTable`를 이용해 토큰을 정수에 매핑시켜보도록 하겠습니다.

먼저 `tf.text.UnicodeScriptTokenizer`로 토큰화기를 만들겠습니다. 

In [39]:
tokenizer = tf_text.UnicodeScriptTokenizer()

이제 데이터를 소문자화 및 토큰화하는 함수를 정의합니다.

In [40]:
def tokenize(text, unused_label):
    lower_case = tf_text.case_fold_utf8(text)
    return tokenizer.tokenize(lower_case)

`tf.data.Dataset.map`을 이용해 데이터세트를 토큰화하겠습니다.

In [41]:
tokenized_ds = all_labeled_data.map(tokenize)

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


토큰화된 데이터세트 중 5개를 살펴보겠습니다.

In [42]:
for text_batch in tokenized_ds.take(5):
    print("Tokens: ", text_batch.numpy())

Tokens:  [b'the' b'dust' b',' b'put' b'on' b'their' b'tunics' b'.' b'then' b'again']
Tokens:  [b'but' b'miss' b"'" b'd' b'patroclus' b';' b'the' b'innocuous' b'point'
 b',']
Tokens:  [b'he' b'said' b';' b'and' b'from' b'th' b"'" b'applauding' b'ranks' b'of'
 b'greece']
Tokens:  [b'the' b'gods' b',' b'or' b'our' b'supineness' b',' b'succor' b'troy'
 b'.']
Tokens:  [b'and' b'struck' b'the' b'circle' b'of' b'aeneas' b"'" b'shield']


다음으로, 단어의 언급 빈도수에 따라 토큰을 정렬하고, `VOCAB_SIZE` 제한을 유지하면서 단어 사전을 빌드할 것입니다.

In [43]:
tokenized_ds = configure_dataset(tokenized_ds)

vocab_dict = collections.defaultdict(lambda: 0)
for toks in tokenized_ds.as_numpy_iterator():
    for tok in toks:
        vocab_dict[tok] += 1
# 단어의 언급 빈도수에 따라 정렬
vocab = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
vocab = [token for token, count in vocab]
vocab = vocab[:VOCAB_SIZE]  # 가장 많이 언급되는 단어부터 최대 10000개의 단어까지
vocab_size = len(vocab)
print("Vocab size: ", vocab_size)
print("First five vocab entries: ", vocab[:5])

Vocab size:  10000
First five vocab entries:  [b',', b'the', b'and', b"'", b'of']


토큰을 정수로 변환하기 위해서 `StaticVocabularyTable`을 생성하는 vocab 세트를 이용합니다. 토큰을 정수 범위 [2, vocab_size + 2]로 변환할 것입니다. 0과 1을 빼는 이유는 0은 패딩을 위한 값, 1은 out-of-vocabulary(OOV)를 위한 값으로 사용될 것이기 때문입니다.

In [44]:
keys = vocab  # 단어
values = range(2, len(vocab) + 2)  # 정수 : 2 ~ 10,002
# 단어와 정수를 대응시키기
init = tf.lookup.KeyValueTensorInitializer(
    keys, values, key_dtype=tf.string, value_dtype=tf.int64
)

num_oov_buckets = 1  # 사전을 벗어나는 단어는 1로 처리
vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets)  # 룩-업 테이블(사전) 구성

마지막으로, 표준화, 토큰화, 벡터화시키는 함수를 정의합니다.

In [45]:
def preprocess_text(text, label):
    standardized = tf_text.case_fold_utf8(text)  # 소문자화
    tokenized = tokenizer.tokenize(standardized)  # 토큰화
    vectorized = vocab_table.lookup(tokenized)  # 벡터화
    return vectorized, label

하나의 (example, label) 쌍을 뽑아내서 벡터화까지 된 결과를 살펴봅시다.

In [46]:
example_text, example_label = next(iter(all_labeled_data))
print("Sentence: ", example_text.numpy())
vectorized_text, example_label = preprocess_text(example_text, example_label)
print("Vectorized sentence: ", vectorized_text.numpy())

Sentence:  b'The dust, put on their tunics. Then again'
Vectorized sentence:  [   3  317    2  383   22   30 4110    7   33  161]


`tf.data.Dataset.map`을 통해 벡터화된 데이터세트로 만듭니다.

In [47]:
all_encoded_data = all_labeled_data.map(preprocess_text)

### 데이터세트에서 훈련과 테스트 세트 분리하기

Keras의 `TextVectorization` 레이어를 사용할 때는 패딩 처리된 벡터화된 데이터가 필요했습니다. 하지만 현재 데이터세트는 데이터가 모두 같은 크기와 모양을 가질 필요가 없습니다.

데이터세트를 훈련과 검증 세트로 나누겠습니다. 이후 훈련과 검증 데이터세트를 배치화 시키겠습니다.


In [48]:
train_data = all_encoded_data.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE)
validation_data = all_encoded_data.take(VALIDATION_SIZE)

In [49]:
train_data = train_data.padded_batch(BATCH_SIZE)
validation_data = validation_data.padded_batch(BATCH_SIZE)

이제 `train_data`와 `validation_data`는 (example, label)의 쌍의 집합이 아니라 배치의 집합입니다.

In [50]:
sample_text, sample_labels = next(iter(validation_data))
print("Text batch shape: ", sample_text.shape)
print("Label batch shape: ", sample_labels.shape)
print("First text example: ", sample_text[0])
print("First label example: ", sample_labels[0])

Text batch shape:  (64, 17)
Label batch shape:  (64,)
First text example:  tf.Tensor(
[   3  317    2  383   22   30 4110    7   33  161    0    0    0    0
    0    0    0], shape=(17,), dtype=int64)
First label example:  tf.Tensor(0, shape=(), dtype=int64)


`0`과 `1`은 패딩과 OOV를 위한 토큰 값이므로 `vocab_size`를 2 늘려준다.

In [51]:
vocab_size += 2

데이터셋을 `configure`해서 더 좋은 성능을 가질 수 있도록 합니다.

In [52]:
train_data = configure_dataset(train_data)
validation_data = configure_dataset(validation_data)

### 모델 훈련시키기

위에서 만들어놓았던 모델 생성 함수를 이용해 모델을 만들어 훈련시킬 수 있습니다.


In [53]:
model = create_model(vocab_size=vocab_size, num_labels=3)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)
history = model.fit(train_data, validation_data=validation_data, epochs=3)

Epoch 1/3
697/697 [==============================] - 43s 23ms/step - loss: 0.5245 - accuracy: 0.7625 - val_loss: 0.3878 - val_accuracy: 0.8354
Epoch 2/3
697/697 [==============================] - 10s 14ms/step - loss: 0.2855 - accuracy: 0.8849 - val_loss: 0.3776 - val_accuracy: 0.8438
Epoch 3/3
697/697 [==============================] - 10s 14ms/step - loss: 0.1913 - accuracy: 0.9278 - val_loss: 0.4142 - val_accuracy: 0.8402


In [54]:
loss, accuracy = model.evaluate(validation_data)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

79/79 [==============================] - 1s 2ms/step - loss: 0.4142 - accuracy: 0.8402
Loss:  0.41424307227134705
Accuracy: 84.02%


### 모델 익스포트하기

정제되지 않은 문자열을 모델의 input으로 받고 싶을 경우, `TextVectorization` 레이어를 만들어서 우리가 직접 만들어보았던 전처리 함수의 기능을 수행하도록 할 수 있습니다. 이미 룩업 테이블을 만들면서 단어를 훈련시켜 놓았기 때문에 `adapt`를 수행하는 대신에 `set_vocabulary`를 사용할 수 있습니다.

In [55]:
preprocess_layer = TextVectorization(
    max_tokens=vocab_size,
    standardize=tf_text.case_fold_utf8,
    split=tokenizer.tokenize,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

preprocess_layer.set_vocabulary(vocab)

In [56]:
export_model = tf.keras.Sequential([
                                    preprocess_layer,
                                    model,
                                    layers.Activation('sigmoid')
])

export_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

In [57]:
test_ds = all_labeled_data.take(VALIDATION_SIZE).batch(BATCH_SIZE)
test_ds = configure_dataset(test_ds)
loss, accuracy = export_model.evaluate(test_ds)
print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

/usr/local/lib/python3.7/dist-packages/keras/backend.py:4908: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


79/79 [==============================] - 10s 27ms/step - loss: 0.5812 - accuracy: 0.7808
Loss:  0.5812350511550903
Accuracy: 78.08%


### 새로운 데이터로 실행시켜보기

In [58]:
inputs = [
    "Join'd to th' Ionians with their flowing robes,",  # Label: 1
    "the allies, and his armour flashed about him so that he seemed to all",  # Label: 2
    "And with loud clangor of his arms he fell.",  # Label: 0
]
predicted_scores = export_model.predict(inputs)
predicted_labels = tf.argmax(predicted_scores, axis=1)
for input, label in zip(inputs, predicted_labels):
    print("Question: ", input)
    print("Predicted label: ", label.numpy())

Question:  Join'd to th' Ionians with their flowing robes,
Predicted label:  1
Question:  the allies, and his armour flashed about him so that he seemed to all
Predicted label:  2
Question:  And with loud clangor of his arms he fell.
Predicted label:  0


### TensorFlow Datasets(TFDS)를 사용해 더 많은 데이터세트 다운로드하기

[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)에서 더 많은 데이터세트를 다운로드 할 수 있습니다. [IMDB Large Movie Review dataset](https://www.tensorflow.org/datasets/catalog/imdb_reviews)를 다운로드 받고 이진 분류를 위한 모델 훈련에 사용해봅시다.

In [59]:
train_ds = tfds.load(
    'imdb_reviews',
    split='train[:80%]',
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True
)

In [60]:
val_ds = tfds.load(
    'imdb_reviews',
    split='train[80%:]',
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True
)

In [60]:
for review_batch, label_batch in val_ds.take(1):
    for i in range(5):
        print("Review: ", review_batch[i].numpy())
        print("Label: ", label_batch[i].numpy())

### 훈련을 위한 데이터세트 준비하기

In [63]:
vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH
)

train_text = train_ds.map(lambda text, labels: text)
vectorize_layer.adapt(train_text)

In [64]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [65]:
train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)

In [66]:
train_ds = configure_dataset(train_ds)
val_ds = configure_dataset(val_ds)

### 모델 훈련시키기

In [67]:
model = create_model(vocab_size=VOCAB_SIZE+1, num_labels=1)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          640064    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d_2 (Glo  (None, 64)                0         
balMaxPooling1D)                                                 
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 660,673
Trainable params: 660,673
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

In [69]:
history = model.fit(train_ds, validation_data=val_ds, epochs=3)

Epoch 1/3
313/313 [==============================] - 20s 60ms/step - loss: 0.5480 - accuracy: 0.6554 - val_loss: 0.3790 - val_accuracy: 0.8250
Epoch 2/3
313/313 [==============================] - 15s 49ms/step - loss: 0.3025 - accuracy: 0.8660 - val_loss: 0.3209 - val_accuracy: 0.8586
Epoch 3/3
313/313 [==============================] - 16s 50ms/step - loss: 0.1856 - accuracy: 0.9269 - val_loss: 0.3264 - val_accuracy: 0.8634


In [70]:
loss, accuracy = model.evaluate(val_ds)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

79/79 [==============================] - 1s 13ms/step - loss: 0.3264 - accuracy: 0.8634
Loss:  0.32642149925231934
Accuracy: 86.34%


### 모델 익스포트하기

In [71]:
export_model = tf.keras.Sequential([
                                    vectorize_layer,
                                    model,
                                    layers.Activation('sigmoid')
])

export_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

In [72]:
inputs = [
    "This is a fantastic movie.",
    "This is a bad movie.",
    "This movie was so bad that it was good.",
    "I will never say yes to watching this movie."
]

predicted_scores = export_model.predict(inputs)
predicted_labels = [int(round(x[0])) for x in predicted_scores]
for input, label in zip(inputs, predicted_labels):
    print("Question: ", input)
    print("Predicted label: ", label)

Question:  This is a fantastic movie.
Predicted label:  1
Question:  This is a bad movie.
Predicted label:  0
Question:  This movie was so bad that it was good.
Predicted label:  0
Question:  I will never say yes to watching this movie.
Predicted label:  0
